In [1]:
# Import necessary libraries
import pandas as pd

# Load dataset
df = pd.read_csv("spam.csv", encoding="latin-1")

# Keep only relevant columns
df = df[['v1', 'v2']]
df.columns = ['label', 'message']

# Convert labels to binary (spam = 1, ham = 0) for machine learning models
df['label'] = df['label'].map({'ham': 0, 'spam': 1})

# Display the first few rows of the dataset
print(df.head())

# Display the count of spam and non-spam messages
print(df['label'].value_counts())


   label                                            message
0      0  Go until jurong point, crazy.. Available only ...
1      0                      Ok lar... Joking wif u oni...
2      1  Free entry in 2 a wkly comp to win FA Cup fina...
3      0  U dun say so early hor... U c already then say...
4      0  Nah I don't think he goes to usf, he lives aro...
label
0    4825
1     747
Name: count, dtype: int64


In [2]:
# Step 1: Load Required Libraries for Text Processing
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Download necessary NLTK resources
nltk.download('stopwords')
nltk.download('wordnet')

# Initialize the lemmatizer and stopwords list
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Define a function to clean and preprocess SMS messages
def preprocess_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'\W', ' ', text)  # Remove special characters
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    words = text.split()
     # Lemmatize words and remove stopwords
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return ' '.join(words)


# Apply the preprocessing function to all messages in the dataset
df['processed_message'] = df['message'].apply(preprocess_text)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:

# Step 2: Convert Text Data into Numerical Form Using TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer


# Convert text into TF-IDF vectors (max 5000 features)
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(df['processed_message']).toarray()
y = df['label']


In [4]:
# Step 3: Split Data into Training and Testing Sets
from sklearn.model_selection import train_test_split

# 80% training, 20% testing split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


In [5]:
# Step 4: Train a Machine Learning Model (Logistic Regression)
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Initialize and train the logistic regression model
model = LogisticRegression()
model.fit(X_train, y_train)


LogisticRegression()

In [6]:
# Step 5: Make Predictions on Test Data
y_pred = model.predict(X_test)

In [7]:
# Step 6: Evaluate the Model Performance
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.9650224215246637
              precision    recall  f1-score   support

           0       0.96      1.00      0.98       966
           1       0.99      0.74      0.85       149

    accuracy                           0.97      1115
   macro avg       0.98      0.87      0.92      1115
weighted avg       0.97      0.97      0.96      1115



In [8]:
# Step 7: Save the Trained Model and TF-IDF Vectorizer
import pickle

# Save the trained model to a file
pickle.dump(model, open("spam_classifier.pkl", "wb"))

# Save the vectorizer for future text transformation
pickle.dump(vectorizer, open("tfidf_vectorizer.pkl", "wb"))


In [9]:
# Step 8: Load the Saved Model and Vectorizer for Prediction
import pickle

# Load the saved model and vectorizer
model = pickle.load(open("spam_classifier.pkl", "rb"))
vectorizer = pickle.load(open("tfidf_vectorizer.pkl", "rb"))

# Define a function to classify a new SMS message
def predict_message(message):
    # Preprocess the input message
    message = preprocess_text(message)  # Use the same preprocessing function
    message_vector = vectorizer.transform([message]).toarray()
    
    # Predict
    prediction = model.predict(message_vector)[0]
    
    return "Spam" if prediction == 1 else "Not Spam"

# Take input from user
while True:
    user_input = input("\nEnter an SMS to check (or type 'exit' to stop): ")
    if user_input.lower() == 'exit':
        break
    result = predict_message(user_input)
    print(f"Prediction: {result}")


Prediction: Not Spam
